In [104]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import wrds
import seaborn as sns
from scipy.stats import ttest_ind


# Data Generation

In [105]:
df=pd.read_csv('data.csv',sep=';')
df.head()

,permno,date,Rn,shrout,prc,siccd,mcap,mcap_l,rf,Rm,const,Rn_e,Rm_e
0,10001,2009-12-31,0.162621,4361.0,10.3000,4925,44918.3000,43871.6600,0.000032,0.028475,1,0.162589,0.028443
1,10001,2010-01-29,-0.018932,4361.0,10.0600,4925,43871.6600,43646.6324,0.000013,-0.037172,1,-0.018945,-0.037185
2,10001,2010-02-26,-0.000656,4361.0,10.0084,4925,43646.6324,44351.3700,0.000061,0.034744,1,-0.000717,0.034683
3,10001,2010-03-31,0.020643,4361.0,10.1700,4925,44351.3700,69137.3000,0.000112,0.063668,1,0.020531,0.063556
4,10001,2010-04-30,0.124385,6070.0,11.3900,4925,69137.3000,69209.4000,0.000118,0.020036,1,0.124267,0.019918


In [106]:
start_date = '1964-01-01'
end_date = '2023-12-31'
Industry_Returns = pd.read_csv('12_Industry_Portfolios.CSV',index_col='date')
Industry_Returns = Industry_Returns[(Industry_Returns.index >= start_date) & (Industry_Returns.index <= end_date)]
Industry_Returns.index = pd.to_datetime(Industry_Returns.index, format='%Y%m', errors='coerce')+pd.offsets.MonthEnd(0)
Industry_Returns.head()

,NoDur,Durbl,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
date,,,,,,,,,,,,
1964-01-31,0.8,1.22,2.49,4.85,3.1,2.99,3.32,1.4,2.52,3.7,-1.18,2.77
1964-02-29,1.86,1.64,3.11,1.08,1.98,1.91,-0.28,0.87,1.63,1.65,3.85,6.37
1964-03-31,3.08,1.08,4.22,1.4,2.61,3.42,-0.9,-0.74,2.43,-1.94,0.68,3.39
1964-04-30,-0.47,1.8,-1.79,4.04,-1.48,-3.79,0.83,1.05,2.88,-1.42,-1.02,1.1
1964-05-31,1.56,1.52,1.58,2.15,2.09,2.83,-0.39,0.84,3.91,0.52,-0.03,3.85


In [107]:
# Get the list of all the SIC codes
with open('Siccodes12.txt', 'r') as file:
    lines = file.readlines()

dic = {}

for j in range(0, 10000):
    dic[j] = 12
    sic = lines[0].strip().split(" ")[0]
    i = 1
    while i < len(lines)-2:
        if lines[i].strip() == "":
            sic = lines[i+1].strip().split(" ")[0]
            i += 2
            continue
        else:
            a = lines[i].strip().split("-")
            if j in range(int(a[0]), int(a[1])+1):
                dic[j] = sic
            i += 1

df_ind = pd.DataFrame(dic.items(), columns=['SIC', 'FF_12'])

df_ind.to_csv('48_Industries.csv', index=False)

In [108]:
df_ind.head()

,SIC,FF_12
0,0,12
1,1,12
2,2,12
3,3,12
4,4,12


In [109]:
#---------------------------------------------
# Industry Definitions  and Returns 
#---------------------------------------------

Industry_Definitions = pd.read_csv('48_Industries.CSV')
Industry_Definitions.rename(columns={'SIC':'siccd'},inplace=True)
Industries = Industry_Returns.columns.tolist()
#num_industries = len(Industries)
Industry_Names = pd.DataFrame(Industries, index=np.arange(1,13 )).reset_index() #num_industries+1
Industry_Names.columns=['Industry','Name']
Industries

['NoDur',
 'Durbl',
 'Manuf',
 'Enrgy',
 'Chems',
 'BusEq',
 'Telcm',
 'Utils',
 'Shops',
 'Hlth ',
 'Money',
 'Other']

In [110]:
Industry_Returns.head()

,NoDur,Durbl,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
date,,,,,,,,,,,,
1964-01-31,0.8,1.22,2.49,4.85,3.1,2.99,3.32,1.4,2.52,3.7,-1.18,2.77
1964-02-29,1.86,1.64,3.11,1.08,1.98,1.91,-0.28,0.87,1.63,1.65,3.85,6.37
1964-03-31,3.08,1.08,4.22,1.4,2.61,3.42,-0.9,-0.74,2.43,-1.94,0.68,3.39
1964-04-30,-0.47,1.8,-1.79,4.04,-1.48,-3.79,0.83,1.05,2.88,-1.42,-1.02,1.1
1964-05-31,1.56,1.52,1.58,2.15,2.09,2.83,-0.39,0.84,3.91,0.52,-0.03,3.85


In [111]:
df8=pd.merge(df,Industry_Definitions[['siccd','FF_12']],on=['siccd'], how='left')
df8['date']=pd.to_datetime(df8['date'])
df8.head()

,permno,date,Rn,shrout,prc,siccd,mcap,mcap_l,rf,Rm,const,Rn_e,Rm_e,FF_12
0,10001,2009-12-31,0.162621,4361.0,10.3000,4925,44918.3000,43871.6600,0.000032,0.028475,1,0.162589,0.028443,8
1,10001,2010-01-29,-0.018932,4361.0,10.0600,4925,43871.6600,43646.6324,0.000013,-0.037172,1,-0.018945,-0.037185,8
2,10001,2010-02-26,-0.000656,4361.0,10.0084,4925,43646.6324,44351.3700,0.000061,0.034744,1,-0.000717,0.034683,8
3,10001,2010-03-31,0.020643,4361.0,10.1700,4925,44351.3700,69137.3000,0.000112,0.063668,1,0.020531,0.063556,8
4,10001,2010-04-30,0.124385,6070.0,11.3900,4925,69137.3000,69209.4000,0.000118,0.020036,1,0.124267,0.019918,8


In [112]:
df8 = pd.merge(df8, Industry_Returns,on=['date'], how='left')
# Convert the relevant columns to numeric, coercing errors
for industry in Industries:
    df8[industry] = pd.to_numeric(df8[industry], errors='coerce')

# Convert rf to a numpy array
rf_array = np.array(df8['rf'])

# Perform the arithmetic operation
df8[Industries] = df8[Industries] / 100 - rf_array[:, np.newaxis]

df8.head()

,permno,date,Rn,shrout,prc,siccd,mcap,mcap_l,rf,Rm,...,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
0,10001,2009-12-31,0.162621,4361.0,10.3000,4925,44918.3000,43871.6600,0.000032,0.028475,...,0.046668,-0.005032,-0.003232,0.059768,0.064168,0.058568,0.019468,0.024668,0.003068,0.014668
1,10001,2009-12-31,0.162621,4361.0,10.3000,4925,44918.3000,43871.6600,0.000032,0.028475,...,0.086468,0.090268,0.074068,0.086168,0.102968,0.063168,0.056868,0.081368,0.024968,0.060068
2,10001,2010-01-29,-0.018932,4361.0,10.0600,4925,43871.6600,43646.6324,0.000013,-0.037172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,2010-02-26,-0.000656,4361.0,10.0084,4925,43646.6324,44351.3700,0.000061,0.034744,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,2010-03-31,0.020643,4361.0,10.1700,4925,44351.3700,69137.3000,0.000112,0.063668,...,0.078188,0.032188,0.040788,0.066688,0.076188,0.031088,0.062488,0.036188,0.082988,0.077788


In [113]:
df8[Industries] = pd.get_dummies(df8['FF_12']).replace({True: 1, False: 0})
df8.head()

C:\Users\Yass La Menace\AppData\Local\Temp\ipykernel_27948\417722347.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df8[Industries] = pd.get_dummies(df8['FF_12']).replace({True: 1, False: 0})


,permno,date,Rn,shrout,prc,siccd,mcap,mcap_l,rf,Rm,...,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
0,10001,2009-12-31,0.162621,4361.0,10.3000,4925,44918.3000,43871.6600,0.000032,0.028475,...,0,0,0,0,0,1,0,0,0,0
1,10001,2009-12-31,0.162621,4361.0,10.3000,4925,44918.3000,43871.6600,0.000032,0.028475,...,0,0,0,0,0,1,0,0,0,0
2,10001,2010-01-29,-0.018932,4361.0,10.0600,4925,43871.6600,43646.6324,0.000013,-0.037172,...,0,0,0,0,0,1,0,0,0,0
3,10001,2010-02-26,-0.000656,4361.0,10.0084,4925,43646.6324,44351.3700,0.000061,0.034744,...,0,0,0,0,0,1,0,0,0,0
4,10001,2010-03-31,0.020643,4361.0,10.1700,4925,44351.3700,69137.3000,0.000112,0.063668,...,0,0,0,0,0,1,0,0,0,0


# PF Construction

In [114]:
df8.head()

,permno,date,Rn,shrout,prc,siccd,mcap,mcap_l,rf,Rm,...,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
0,10001,2009-12-31,0.162621,4361.0,10.3000,4925,44918.3000,43871.6600,0.000032,0.028475,...,0,0,0,0,0,1,0,0,0,0
1,10001,2009-12-31,0.162621,4361.0,10.3000,4925,44918.3000,43871.6600,0.000032,0.028475,...,0,0,0,0,0,1,0,0,0,0
2,10001,2010-01-29,-0.018932,4361.0,10.0600,4925,43871.6600,43646.6324,0.000013,-0.037172,...,0,0,0,0,0,1,0,0,0,0
3,10001,2010-02-26,-0.000656,4361.0,10.0084,4925,43646.6324,44351.3700,0.000061,0.034744,...,0,0,0,0,0,1,0,0,0,0
4,10001,2010-03-31,0.020643,4361.0,10.1700,4925,44351.3700,69137.3000,0.000112,0.063668,...,0,0,0,0,0,1,0,0,0,0


## BAB PF

In [115]:
def compute_betas(df3):
#Drop NaNs
    df3=df3.dropna(subset=['mcap_l','Rn_e','Rm_e']).copy()

    #Remove scarce df3 
    df3['N'] = df3.groupby(['permno'])['date'].transform('count')
    df3 = df3[df3['N']>60].copy()

    #Estimate Rolling Betas
    cov_nm = df3.set_index('date').groupby('permno')[['Rn_e','Rm_e']].rolling(60, min_periods=36).cov()
    #beta_n = (cov_nm.iloc[1::2,1].droplevel(2)/cov_nm.iloc[0::2,1].droplevel(2))
    beta_n = (cov_nm.iloc[0::2,1].droplevel(2)/cov_nm.iloc[1::2,1].droplevel(2))
    beta_n = beta_n.dropna().reset_index().rename(columns={'Rm_e':'beta'})

    #reconstruct df
    beta_n['date'] = beta_n['date'] + pd.DateOffset(months=1)
    df3 = pd.merge(df3, beta_n, on=['date','permno'], how='left')

    #Winsorsize
    df3['beta'] = df3['beta'].clip(df3['beta'].quantile(0.05),df3['beta'].quantile(0.95))

    #Show first 5 rows
    return df3

In [116]:
def construct_BAB(df3):
    # Weights
    df3['z'] = df3.groupby('date')['beta'].transform(lambda x: x.rank())
    df3['z_'] = df3['z']-df3.groupby('date')['z'].transform('mean')
    df3['k'] = np.abs(df3['z_'])
    df3['k'] = 2/df3.groupby('date')['k'].transform('sum')
    df3['w_H'] = df3['k'] * df3['z_'] * (df3['z_']>0) 
    df3['w_L'] = -df3['k'] * df3['z_'] * (df3['z_']<0)

    # Weighted returns and beta
    df3['beta_H'] = df3['w_H'] * df3['beta']
    df3['beta_L'] = df3['w_L'] * df3['beta']
    df3['R_H'] = df3['w_H'] * df3['Rn']
    df3['R_L'] = df3['w_L'] * df3['Rn']
    df3['R_H_e'] = df3['w_H'] * df3['Rn_e']
    df3['R_L_e'] = df3['w_L'] * df3['Rn_e']
    BAB = df3.groupby('date')[['R_H','R_L','R_H_e','R_L_e','beta_H','beta_L']].sum().reset_index()
    BAB['BAB2'] = BAB['R_L_e']/BAB['beta_L'] - BAB['R_H_e']/BAB['beta_H']

    return BAB


# MOM

In [117]:
def construct_mom(df):
    df=df.sort_values(['permno','date']).reset_index(drop=True)
    df['LN_Cret12'] = df.groupby('permno').apply(lambda x: np.log(x.Rn+1).rolling(11,closed='left').sum()).values
    df['Rn_f'] = df.groupby(['permno'])['Rn'].shift(-1)
    df4=df.copy()
    #Momentum decile weights
    q = 10
    #Momentum is the decile
    df4['Momentum'] = df4.groupby('date')['LN_Cret12'].transform(lambda x: pd.qcut(x,q,labels=False, duplicates='drop'))
    df4['w_L'] = (df4['Momentum'].isin([q - 1, q - 2, q - 3]))*df4['mcap']
    df4['w_S'] = (df4['Momentum'].isin([0, 1, 2]))*df4['mcap']
    #Normalize the weights
    df4['w_L'] = df4['w_L']/df4.groupby(['date'])['w_L'].transform('sum')
    df4['w_S'] = df4['w_S']/df4.groupby(['date'])['w_S'].transform('sum')
    df4['w_Mom'] = df4['w_L'] - df4['w_S']
    Momentum = df4.groupby('date').apply(lambda x: np.sum(x['Rn_f']*x['w_Mom'])).reset_index()
    Momentum['date'] =  Momentum['date'] + pd.DateOffset(months=1)
    Momentum.columns = ['date','Mom_ret']
    return Momentum

# IV

In [118]:
df_idio=pd.read_csv('idio_vol.csv')
df_idio.head()

,permno,date,idio_vol
0,10001,2010-05-28,8.073456e-17
1,10001,2010-04-30,1.602407e-01
2,10001,2010-03-31,2.125114e-01
3,10001,2010-02-26,1.990379e-01
4,10001,2010-01-29,1.815296e-01


In [119]:
def construct_IV_df(df5,df_idio):
    df_idio['date']=pd.to_datetime(df_idio['date'])
    df5=df5.merge(df_idio,on=['date','permno'])
    df5['idio_vol'] = df5['idio_vol'].clip(df5['idio_vol'].quantile(0.05), df5['idio_vol'].quantile(0.95))
    df5['Rn_f'] = df5.groupby(['permno'])['Rn'].shift(-1)
    return df5


In [120]:
def comnstruct_IV(df5):
    df5=df5.dropna(subset=['idio_vol']).copy()
    df5['idio_q']=df5.groupby('date')['idio_vol'].transform(lambda x :  pd.qcut(x, 10,labels=False, duplicates='drop'))
    df_deciles=df5[df5['idio_q'].isin([0,1,2,7,8,9])].copy()
    df_deciles['w_L'] = (df_deciles['idio_q'].isin([7,8,9]))*df_deciles['mcap']
    df_deciles['w_S'] = (df_deciles['idio_q'].isin([0,1,2]))*df_deciles['mcap']
    #Normalize the weights
    df_deciles['w_L'] = df_deciles['w_L']/df_deciles.groupby(['date'])['w_L'].transform('sum')
    df_deciles['w_S'] = df_deciles['w_S']/df_deciles.groupby(['date'])['w_S'].transform('sum')
    df_deciles['w_iv'] = df_deciles['w_L'] - df_deciles['w_S']
    idio = df_deciles.groupby('date').apply(lambda x: np.sum(x['Rn_f']*x['w_iv'])).reset_index()
    idio['date'] =  idio['date'] + pd.DateOffset(months=1)
    idio.columns = ['date','idio_ret']

    return idio

In [121]:
df_iv=construct_IV_df(df8[df8['Manuf']==1],df_idio)
test_iv=comnstruct_IV(df_iv)
test_iv

C:\Users\Yass La Menace\AppData\Local\Temp\ipykernel_27948\2118778774.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  idio = df_deciles.groupby('date').apply(lambda x: np.sum(x['Rn_f']*x['w_iv'])).reset_index()


,date,idio_ret
0,1964-02-29,-0.009368
1,1964-03-28,0.020016
2,1964-04-30,0.023156
3,1964-05-30,0.013724
4,1964-06-28,-0.032643
...,...,...
714,2023-09-30,0.031120
715,2023-10-29,-0.079715
716,2023-11-30,0.009270
717,2023-12-30,0.041010


# Risk Parity

In [122]:
df_beta=compute_betas(df8[df8['Manuf']==1])
BAB=construct_BAB(df_beta)
BAB.head()

,date,R_H,R_L,R_H_e,R_L_e,beta_H,beta_L,BAB2
0,1964-01-31,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1964-02-28,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1964-03-31,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1964-04-30,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1964-05-28,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [123]:
Momentum=construct_mom(df8[df8['Manuf']==1])
Momentum.head()

C:\Users\Yass La Menace\AppData\Local\Temp\ipykernel_27948\1443563410.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['LN_Cret12'] = df.groupby('permno').apply(lambda x: np.log(x.Rn+1).rolling(11,closed='left').sum()).values
C:\Users\Yass La Menace\AppData\Local\Temp\ipykernel_27948\1443563410.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Momentum = df4.groupby('date').apply(lambda x: np.sum(x

,date,Mom_ret
0,1964-02-29,0.0
1,1964-03-28,0.0
2,1964-04-30,0.0
3,1964-05-30,0.0
4,1964-06-28,0.0


In [124]:
df_iv=construct_IV_df(df8[df8['Manuf']==1],df_idio)
idio=comnstruct_IV(df_iv)
idio

C:\Users\Yass La Menace\AppData\Local\Temp\ipykernel_27948\2118778774.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  idio = df_deciles.groupby('date').apply(lambda x: np.sum(x['Rn_f']*x['w_iv'])).reset_index()


,date,idio_ret
0,1964-02-29,-0.009368
1,1964-03-28,0.020016
2,1964-04-30,0.023156
3,1964-05-30,0.013724
4,1964-06-28,-0.032643
...,...,...
714,2023-09-30,0.031120
715,2023-10-29,-0.079715
716,2023-11-30,0.009270
717,2023-12-30,0.041010


In [125]:
def construct_strat_df(BAB,Momentum,idio):
    r_bab=BAB.copy()
    r_mom=Momentum.groupby('date')['Mom_ret'].sum().copy()
    r_iv=idio.groupby('date')['idio_ret'].sum().copy()
    r_strat=r_bab.merge(r_mom,on='date').merge(r_iv,on='date')
    r_strat.rename(columns={'BAB2':'ret_bab',	'Mom_ret':'ret_mom',	'idio_ret':'ret_iv'},inplace=True)
    r_strat=r_strat[['date','ret_bab', 'ret_mom', 'ret_iv']]
    return r_strat

In [126]:
def risk_parity(r_strat):
    window_size=60
    min_req=36
    # Calculate rolling window volatility for each column
    r_strat['volatility_bab'] = r_strat['ret_bab'].rolling(window=window_size,min_periods=min_req).std()
    r_strat['volatility_mom'] = r_strat['ret_mom'].rolling(window=window_size,min_periods=min_req).std()
    r_strat['volatility_iv'] = r_strat['ret_iv'].rolling(window=window_size,min_periods=min_req).std()

    r_strat['total_vol']=(1/r_strat['volatility_bab'])+ (1/r_strat['volatility_mom' ])+ (1/r_strat['volatility_iv'])

    r_strat['w_bab']=(1/r_strat['volatility_bab'])/r_strat['total_vol']    
    r_strat['w_mom']=(1/r_strat['volatility_mom' ])/r_strat['total_vol']    
    r_strat['w_iv']=(1/r_strat['volatility_iv'])/r_strat['total_vol']

    r_strat['ret_rp']=r_strat['ret_bab']*r_strat['w_bab']+r_strat['ret_iv']*r_strat['w_iv']+r_strat['ret_mom']*r_strat['w_mom']
    return r_strat

In [127]:
r_strat=construct_strat_df(BAB,Momentum,idio)
rp=risk_parity(r_strat)
rp.head()

,date,ret_bab,ret_mom,ret_iv,volatility_bab,volatility_mom,volatility_iv,total_vol,w_bab,w_mom,w_iv,ret_rp
0,1964-04-30,NaN,0.000000,0.023156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1964-08-31,NaN,-0.001150,0.013014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1964-09-30,NaN,0.017042,-0.000347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1964-10-30,NaN,0.002262,-0.000131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1964-11-30,NaN,0.018424,-0.024728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
def statistics(r_strat,industry):
    mean=r_strat['ret_rp'].mean()
    std=r_strat['ret_rp'].std()
    sr=mean/std
    t_stat,_=ttest_ind(r_strat['ret_rp'].dropna()*12,0)
    return{'mean':mean,
           'deviation': std,
           'Sharpe_ratio':sr,
           'T-Stat':t_stat,
           'Industry':industry}

# Pipeline

In [129]:
len(Industries)

12

In [130]:
data_final=[]
rps=[]
for ind in Industries:
    df_curr=df8[df8[ind]==1]
    df_beta=compute_betas(df_curr)
    BAB=construct_BAB(df_beta)
    Momentum=construct_mom(df_curr)
    df_iv=construct_IV_df(df_curr,df_idio)
    idio=comnstruct_IV(df_iv)
    r_strat=construct_strat_df(BAB,Momentum,idio)
    rp=risk_parity(r_strat)
    rps.append(rp[['date','ret_rp']])
    data_final.append(statistics(rp,ind))

C:\Users\Yass La Menace\AppData\Local\Temp\ipykernel_27948\1443563410.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['LN_Cret12'] = df.groupby('permno').apply(lambda x: np.log(x.Rn+1).rolling(11,closed='left').sum()).values
C:\Users\Yass La Menace\AppData\Local\Temp\ipykernel_27948\1443563410.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Momentum = df4.groupby('date').apply(lambda x: np.sum(x

In [131]:
rp

,date,ret_bab,ret_mom,ret_iv,volatility_bab,volatility_mom,volatility_iv,total_vol,w_bab,w_mom,w_iv,ret_rp
0,1964-04-30,NaN,0.000000,0.252288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1964-08-31,NaN,-0.023316,0.027199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1964-09-30,NaN,-0.011128,0.078805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1964-10-30,NaN,0.029715,0.117768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1964-11-30,NaN,-0.036861,0.032335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
255,2022-12-30,-0.060151,-0.026003,-0.013543,0.043426,0.047785,0.047061,65.203987,0.353166,0.320945,0.325889,-0.034002
256,2023-02-28,-0.020869,0.023904,0.011019,0.043624,0.047589,0.044886,66.215048,0.346189,0.317351,0.336460,0.004069
257,2023-06-30,0.040427,0.041334,0.011132,0.043451,0.048269,0.044616,66.145206,0.347936,0.313209,0.338854,0.030784
258,2023-08-31,-0.002120,0.078382,-0.056785,0.043219,0.049919,0.044964,65.410026,0.353736,0.306257,0.340007,0.003948


In [132]:
rps

[          date    ret_rp
 0   1964-04-30       NaN
 1   1964-08-31       NaN
 2   1964-09-30       NaN
 3   1964-10-30       NaN
 4   1964-11-30       NaN
 ..         ...       ...
 255 2022-12-30 -0.037583
 256 2023-02-28  0.007394
 257 2023-06-30 -0.026502
 258 2023-08-31 -0.002417
 259 2023-11-30  0.022448
 
 [260 rows x 2 columns],
           date    ret_rp
 0   1964-04-30       NaN
 1   1964-08-31       NaN
 2   1964-09-30       NaN
 3   1964-10-30       NaN
 4   1964-11-30       NaN
 ..         ...       ...
 255 2022-12-30 -0.052532
 256 2023-02-28  0.031691
 257 2023-06-30  0.021132
 258 2023-08-31  0.001056
 259 2023-11-30 -0.005790
 
 [260 rows x 2 columns],
           date    ret_rp
 0   1964-04-30       NaN
 1   1964-08-31       NaN
 2   1964-09-30       NaN
 3   1964-10-30       NaN
 4   1964-11-30       NaN
 ..         ...       ...
 255 2022-12-30 -0.047557
 256 2023-02-28  0.021477
 257 2023-06-30  0.040536
 258 2023-08-31 -0.013669
 259 2023-11-30  0.049416
 
 [260 ro

In [133]:

industries = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth', 'Money', 'Other']

# Combine the list of DataFrames into a single DataFrame and rename columns
df_rps = rps[0].rename(columns={'ret_rp': industries[0]})

for i, df in enumerate(rps[1:], start=1):
    df_rps = pd.merge(df_rps, df.rename(columns={'ret_rp': industries[i]}), on='date', how='outer')

# Set 'date' as the index
df_rps.set_index('date', inplace=True)

# Display the resulting DataFrame
df_rps

,NoDur,Durbl,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
date,,,,,,,,,,,,
1964-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1964-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1964-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1964-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1964-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30,-0.037583,-0.052532,-0.047557,-0.042988,-0.066345,-0.022742,-0.041817,-0.034766,-0.049333,-0.000988,-0.032750,-0.034002
2023-02-28,0.007394,0.031691,0.021477,0.005936,0.011296,0.004737,0.004594,0.013693,-0.019415,-0.065618,0.008369,0.004069
2023-06-30,-0.026502,0.021132,0.040536,0.012936,0.024789,0.019235,-0.000527,0.035165,0.040734,-0.014201,0.028547,0.030784


In [134]:
df_final=pd.DataFrame(data_final)

In [135]:
df_rps['STRAT'] = df_rps.mean(axis=1, skipna=True)

In [136]:
std_s=df_rps['STRAT'].std()*np.sqrt(12)
mean_s=df_rps['STRAT'].mean()*12
SR_s=mean_s/std_s

portfolio_metrics = {
    "Portfolio Mean": mean_s,
    "Portfolio Standard Deviation": std_s,
    "Portfolio Sharpe Ratio": SR_s
}

print(portfolio_metrics)

{'Portfolio Mean': 0.0065083714779890795, 'Portfolio Standard Deviation': 0.07363010519356877, 'Portfolio Sharpe Ratio': 0.08839280428676549}


In [137]:
df_final

,mean,deviation,Sharpe_ratio,T-Stat,Industry
0,-0.000642,0.032744,-0.019594,-0.019549,NoDur
1,-0.000045,0.037292,-0.001206,-0.001203,Durbl
2,0.002169,0.025893,0.083777,0.083585,Manuf
3,0.000449,0.044382,0.010113,0.010090,Enrgy
4,0.002140,0.030563,0.070015,0.069854,Chems
5,0.001759,0.036111,0.048724,0.048612,BusEq
6,0.001994,0.041292,0.048283,0.048172,Telcm
7,0.000358,0.023654,0.015156,0.015121,Utils
8,0.000087,0.030620,0.002828,0.002822,Shops
9,-0.000258,0.048530,-0.005320,-0.005308,Hlth


In [138]:
df_rps.reset_index(inplace=True)

In [150]:
Strat=df_rps[['date','STRAT',]]
Strat['date']=pd.to_datetime(Strat['date'])
df9=pd.read_csv('df7.csv')
df9['date']=pd.to_datetime(df9['date'])
df9=pd.merge(Strat, df9,on=['date'], how='left')
df9.dropna(inplace=True)
df9

C:\Users\Yass La Menace\AppData\Local\Temp\ipykernel_27948\1055468619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Strat['date']=pd.to_datetime(Strat['date'])


,date,STRAT,permno,Rn,shrout,siccd,prc,mcap,mcap_l,rf,...,Shops,Hlth,Money,Other,Mkt-RF,SMB,HML,RMW,CMA,RF
96875,1974-09-30,-0.011442,10006.0,0.032787,5606.0,3743.0,31.500,176589.00,203217.50,0.005142,...,-0.127942,-0.160742,-0.076342,-0.124642,-0.1177,0.0148,0.0558,-0.0444,0.0591,0.81
96876,1974-09-30,-0.011442,10006.0,0.032787,5606.0,3743.0,31.500,176589.00,203217.50,0.005142,...,-0.096542,-0.122942,-0.056442,-0.087042,-0.1177,0.0148,0.0558,-0.0444,0.0591,0.81
96877,1974-09-30,-0.011442,10014.0,0.000000,5460.0,3714.0,1.375,7507.50,8190.00,0.005142,...,-0.127942,-0.160742,-0.076342,-0.124642,-0.1177,0.0148,0.0558,-0.0444,0.0591,0.81
96878,1974-09-30,-0.011442,10014.0,0.000000,5460.0,3714.0,1.375,7507.50,8190.00,0.005142,...,-0.096542,-0.122942,-0.056442,-0.087042,-0.1177,0.0148,0.0558,-0.0444,0.0591,0.81
96879,1974-09-30,-0.011442,10057.0,-0.013699,3845.0,3541.0,9.000,34605.00,33643.75,0.005142,...,-0.127942,-0.160742,-0.076342,-0.124642,-0.1177,0.0148,0.0558,-0.0444,0.0591,0.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726169,2022-11-30,-0.011505,93374.0,0.084325,103542.0,6361.0,54.650,5658570.30,5419388.28,0.003320,...,0.042080,-0.039320,0.027280,-0.004920,0.0460,-0.0274,0.0138,0.0638,0.0318,0.29
726170,2022-11-30,-0.011505,93423.0,0.080269,83157.0,7389.0,24.090,2003252.13,1933400.25,0.003320,...,0.035180,0.050180,0.044580,0.076180,0.0460,-0.0274,0.0138,0.0638,0.0318,0.29
726171,2022-11-30,-0.011505,93423.0,0.080269,83157.0,7389.0,24.090,2003252.13,1933400.25,0.003320,...,0.042080,-0.039320,0.027280,-0.004920,0.0460,-0.0274,0.0138,0.0638,0.0318,0.29
726172,2022-11-30,-0.011505,93426.0,0.202074,12551.0,3676.0,40.570,509194.07,485096.15,0.003320,...,0.035180,0.050180,0.044580,0.076180,0.0460,-0.0274,0.0138,0.0638,0.0318,0.29


In [140]:
df9

,date,STRAT,NoDur,Durbl,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
0,1964-04-30,NaN,0,0,1,0,0,0,0,0,0,0,0,0
1,1964-04-30,NaN,0,0,1,0,0,0,0,0,0,0,0,0
2,1964-04-30,NaN,0,1,0,0,0,0,0,0,0,0,0,0
3,1964-04-30,NaN,0,1,0,0,0,0,0,0,0,0,0,0
4,1964-04-30,NaN,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951489,2023-11-30,0.022851,0,0,1,0,0,0,0,0,0,0,0,0
951490,2023-11-30,0.022851,0,0,0,0,0,0,0,0,1,0,0,0
951491,2023-11-30,0.022851,0,0,0,0,0,0,0,0,0,0,1,0
951492,2023-11-30,0.022851,0,0,0,0,0,0,0,0,0,0,0,1


In [151]:
import pandas as pd
import numpy as np
import statsmodels.api as sm




# Define the list of industry and Fama-French columns
Industries = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth', 'Money', 'Other']
Fama_French = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

# Strip any leading or trailing spaces from column names
df9.columns = df9.columns.str.strip()


if 'date' not in df9.columns:
    raise KeyError("'date' column not found in df7")
if 'date' not in Strat.columns:
    raise KeyError("'date' column not found in Strat")

# Merge the dataframes on 'date'
tmp = pd.merge(Strat, df9[['date'] + Industries + Fama_French].drop_duplicates(), on='date', how='left')

# Add a constant column for the regression
tmp['const'] = 1

# Drop rows with any NaN values
tmp = tmp.dropna()


# Industry Exposure
RegOLS = sm.OLS(tmp['STRAT'], tmp[['const'] + Industries + Fama_French]).fit()

# Print regression parameters and R-squared value
print(f"Regression parameters:\n{pd.concat([RegOLS.params, RegOLS.tvalues], axis=1)}\n")
print(f'R^2: {RegOLS.rsquared}')


Regression parameters:
               0         1
const  -0.002752 -2.823583
NoDur   0.073743  1.413664
Durbl   0.016554  0.705788
Manuf  -0.092152 -1.504522
Enrgy  -0.047065 -3.417006
Chems  -0.072398 -1.519999
BusEq  -0.013272 -0.389232
Telcm  -0.015212 -0.566920
Utils  -0.001112 -0.036009
Shops  -0.020505 -0.454340
Hlth    0.006151  0.242694
Money  -0.100538 -2.280414
Other   0.315985  5.295132
Mkt-RF  0.096743  1.527354
SMB     0.174411  4.093240
HML     0.062700  1.044744
RMW    -0.038836 -0.766257
CMA     0.012529  0.190469

R^2: 0.4168265075135301
